In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGANGP

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/WGGP_WI_new3/'

In [10]:
datgan = DATWGANGP(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0)

In [11]:
test = datgan.fit(df, graph)

[1104 17:15:12 @DATSGAN.py:155] Found preprocessed data
[1104 17:15:12 @DATSGAN.py:163] Preprocessed data have been loaded!



[1104 17:15:13 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1104 17:15:13 @DATSGANModel.py:211] Creating cell for age (in-edges: 0; ancestors: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1104 17:15:13 @registry.py:126] gen/LSTM/age/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1104 17:15:13 @registry.py:134] gen/LSTM/age/FC output: [500, 100]
[1104 17:15:13 @registry.py:126] gen/LSTM/age/FC2_val input: [500, 100]
[1104 17:15:13 @registry.py:134] gen/LSTM/age/FC2_val output: [500, 6]
[

[1104 17:15:13 @registry.py:126] gen/LSTM/work_status/FC input: [500, 100]
[1104 17:15:13 @registry.py:134] gen/LSTM/work_status/FC output: [500, 100]
[1104 17:15:13 @registry.py:126] gen/LSTM/work_status/FC2 input: [500, 100]
[1104 17:15:13 @registry.py:134] gen/LSTM/work_status/FC2 output: [500, 8]
[1104 17:15:13 @registry.py:126] gen/LSTM/work_status/FC3 input: [500, 8]
[1104 17:15:13 @registry.py:134] gen/LSTM/work_status/FC3 output: [500, 100]
[1104 17:15:13 @DATSGANModel.py:211] Creating cell for hh_income (in-edges: 2; ancestors: 4)
[1104 17:15:13 @registry.py:126] gen/LSTM/concat-hh_income/FC_inputs input: [500, 200]
[1104 17:15:13 @registry.py:134] gen/LSTM/concat-hh_income/FC_inputs output: [500, 100]
[1104 17:15:13 @registry.py:126] gen/LSTM/concat-hh_income/FC_attentions input: [500, 200]
[1104 17:15:13 @registry.py:134] gen/LSTM/concat-hh_income/FC_attentions output: [500, 100]
[1104 17:15:13 @registry.py:126] gen/LSTM/concat-hh_income/FC_lstm_state_0 input: [500, 200]
[11

[1104 17:15:14 @registry.py:134] gen/LSTM/hh_vehicles/FC3 output: [500, 100]
[1104 17:15:14 @DATSGANModel.py:211] Creating cell for hh_bikes (in-edges: 2; ancestors: 6)
[1104 17:15:14 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_inputs input: [500, 200]
[1104 17:15:14 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_inputs output: [500, 100]
[1104 17:15:14 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_attentions input: [500, 200]
[1104 17:15:14 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_attentions output: [500, 100]
[1104 17:15:14 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_lstm_state_0 input: [500, 200]
[1104 17:15:14 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_lstm_state_0 output: [500, 100]
[1104 17:15:14 @registry.py:126] gen/LSTM/concat-hh_bikes/FC_lstm_state_1 input: [500, 200]
[1104 17:15:14 @registry.py:134] gen/LSTM/concat-hh_bikes/FC_lstm_state_1 output: [500, 100]
[1104 17:15:14 @registry.py:126] gen/LSTM/hh_bikes/FC input: [500, 100]
[1104 17:15:14 @registry.py:134] ge

[1104 17:15:18 @base.py:209] Setup callbacks graph ...






[1104 17:15:18 @summary.py:46] [MovingAverageSummary] 5 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1104 17:15:18 @summary.py:93] Summarizing collection 'summaries' of size 6.




[1104 17:15:19 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1104 17:15:21 @base.py:236] Initializing the session ...
[1104 17:15:21 @sessinit.py:114] Restoring checkpoint from ../output/Chicago/WGGP_WI_new3/model\model-104999 ...
INFO:tensorflow:Restoring parameters from ../output/Chicago/WGGP_WI_new3/model\model-104999
[1104 17:15:22 @base.py:243] Graph Finalized.


[1104 17:15:22 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...
[1104 17:15:22 @steps.py:124] Start training with global_step=104999
[1104 17:15:22 @monitor.py:347] Found existing JSON inside ../output/Chicago/WGGP_WI_new3/logs, will appe

 98%|#################################################################################1 |171/175[00:09<00:00,26.30it/s]

...

[1104 18:11:55 @base.py:285] Epoch 999 (global_step 209789) finished, time:6.53 seconds.
[1104 18:11:55 @saver.py:79] Model saved to ../output/Chicago/WGGP_WI_new3/model\model-209789.


[1104 18:11:55 @monitor.py:467] GAN_loss/d_loss: -1.0491
[1104 18:11:55 @monitor.py:467] GAN_loss/g_loss: -0.50651
[1104 18:11:55 @monitor.py:467] GAN_loss/gradient_penalty: 0.013865
[1104 18:11:55 @monitor.py:467] GAN_loss/gradient_rms: 1.0001
[1104 18:11:55 @monitor.py:467] GAN_loss/kl_div: 0.093351
[1104 18:11:55 @monitor.py:467] QueueInput/queue_size: 50
[1104 18:11:55 @base.py:275] Start Epoch 1000 ...


100%|###################################################################################|175/175[00:06<00:00,26.89it/s]

[1104 18:12:02 @base.py:285] Epoch 1000 (global_step 209999) finished, time:6.51 seconds.
[1104 18:12:02 @saver.py:79] Model saved to ../output/Chicago/WGGP_WI_new3/model\model-209999.


[1104 18:12:02 @monitor.py:467] GAN_loss/d_loss: -1.0464
[1104 18:12:02 @monitor.py:467] GAN_loss/g_loss: -0.50939
[1104 18:12:02 @monitor.py:467] GAN_loss/gradient_penalty: 0.013636
[1104 18:12:02 @monitor.py:467] GAN_loss/gradient_rms: 0.998
[1104 18:12:02 @monitor.py:467] GAN_loss/kl_div: 0.0832
[1104 18:12:02 @monitor.py:467] QueueInput/queue_size: 50
[1104 18:12:02 @base.py:289] Training has finished!

[1104 18:12:02 @DATSGANModel.py:211] Creating cell for age (in-edges: 0; ancestors: 0)
[1104 18:12:02 @DATSGANModel.py:211] Creating cell for gender (in-edges: 0; ancestors: 0)
[1104 18:12:02 @DATSGANModel.py:211] Creating cell for travel_dow (in-edges: 0; ancestors: 0)
[1104 18:12:02 @DATSGANModel.py:211] Creating cell for license (in-edges: 1; ancestors: 1)
[1104 18:12:02 @DATSGANModel.py:211] Creating cell for education_level (in-edges: 1; ancestors: 1)
[1104 18:12:03 @DATSGANModel.py:211] Creating cell for work_status (in-edges: 2; ancestors: 3)
[1104 18:12:03 @DATSGANModel.py:2

In [12]:
datgan.save('trained', force=True)

[1104 18:12:53 @DATSGAN.py:362] Model saved successfully.


In [13]:
beepy.beep(6)